In [1]:
## Import Packages
from __future__ import print_function

import numpy as np
import pandas as pd
from itertools import product

#Astro Software
import astropy.units as units
from astropy.coordinates import SkyCoord
from astropy.io import fits

#Plotting Packages
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams

import seaborn as sns

from PIL import Image

#from yt.config import ytcfg
#import yt
#import yt.units as u

#Scattering NN
import torch
import torch.nn.functional as F
from torch import optim
from kymatio.torch import Scattering2D
device = "cpu"

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA, FastICA

import skimage
from skimage import filters

from scipy.optimize import curve_fit
from scipy import linalg
from scipy import stats
from scipy.signal import general_gaussian

#I/O
import h5py
import pickle
import glob
import copy
import time

#Plotting Style
%matplotlib inline
plt.style.use('dark_background')
rcParams['text.usetex'] = False
rcParams['axes.titlesize'] = 20
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['legend.fontsize'] = 12
rcParams['axes.labelsize'] = 20
rcParams['font.family'] = 'sans-serif'

#Threading
torch.set_num_threads=2
from multiprocessing import Pool

import ntpath
def path_leaf(path):
    head, tail = ntpath.split(path)
    out = os.path.splitext(tail)[0]
    return out

def hd5_open(file_name,name):
    f=h5py.File(file_name,'r', swmr=True)
    data = f[name][:]
    f.close()
    return data

from matplotlib.colors import LinearSegmentedColormap
cdict1 = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),

         'blue':  ((0.0, 0.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
blue_red1 = LinearSegmentedColormap('BlueRed1', cdict1,N=5000)

from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

def cm_eval(y_pred, y_true,n_classes):
    cm = np.zeros((n_classes,n_classes))
    for indx in range(y_pred.shape[0]):
        cm[y_true[indx],y_pred[indx]] += 1
    return cm.astype(int)

from scipy import interpolate

In [3]:
mnist_train_y = hd5_open('mnist_train_y.h5','main/data')
mnist_test_y = hd5_open('mnist_test_y.h5','main/data')

In [4]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
import julia
from julia import Main
import julia.Base
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [5]:
%%julia
push!(LOAD_PATH, "/n/holystore01/LABS/finkbeiner_lab/Lab/saydjari/Working/2021_03_09/")
import DHC_2DUtils
using SparseArrays
using HDF5

In [6]:
%%julia
function S1_equiv_matrix(fhash,l_shift)
        # fhash is the filter hash output by fink_filter_hash
        # The output matrix converts an S1 coeff vector to S1iso by
        #   summing over l
        # Matrix is stored in sparse CSC format using SparseArrays.
        # DPF 2021-Feb-18

        # Does hash contain Omega filter?
        Omega   = haskey(fhash, "Omega_index")
        if Omega Ω_ind = fhash["Omega_index"] end

        # unpack fhash
        Nl      = length(fhash["theta_value"])
        Nj      = length(fhash["j_value"])
        Nf      = length(fhash["filt_value"])
        ψ_ind   = fhash["psi_index"]
        ϕ_ind   = fhash["phi_index"]

        # number of iso coefficients
        Niso    = Omega ? Nj+2 : Nj+1
        Mat     = zeros(Int32, Nf, Nf)

        # first J elements of iso
        for j = 1:Nj
            for l = 1:Nl
                λ = ψ_ind[j,l]
                λ1 = ψ_ind[j,mod1(l+l_shift,Nl)]
                Mat[λ1, λ] = 1
            end
        end

        # Next elements are ϕ, Ω
        Mat[ϕ_ind, ϕ_ind] = 1
        if Omega Mat[Ω_ind, Ω_ind] = 1 end

        return sparse(Mat)
    end

function S2_equiv_matrix(fhash,l_shift)
    # fhash is the filter hash output by fink_filter_hash
    # The output matrix converts an S2 coeff vector to S2iso by
    #   summing over l1,l2 and fixed Δl.
    # Matrix is stored in sparse CSC format using SparseArrays.
    # DPF 2021-Feb-18

    # Does hash contain Omega filter?
    Omega   = haskey(fhash, "Omega_index")
    if Omega Ω_ind = fhash["Omega_index"] end

    # unpack fhash
    Nl      = length(fhash["theta_value"])
    Nj      = length(fhash["j_value"])
    Nf      = length(fhash["filt_value"])
    ψ_ind   = fhash["psi_index"]
    ϕ_ind   = fhash["phi_index"]

    # number of iso coefficients
    Niso    = Omega ? Nj*Nj*Nl+4*Nj+4 : Nj*Nj*Nl+2*Nj+1
    Mat     = zeros(Int32, Nf*Nf, Nf*Nf)

    # first J*J*L elements of iso
    for j1 = 1:Nj
        for j2 = 1:Nj
            for l1 = 1:Nl
                for l2 = 1:Nl
                    λ1     = ψ_ind[j1,l1]
                    λ2     = ψ_ind[j2,l2]
                    λ1_new     = ψ_ind[j1,mod1(l1+l_shift,Nl)]
                    λ2_new     = ψ_ind[j2,mod1(l2+l_shift,Nl)]

                    Icoeff = λ1+Nf*(λ2-1)
                    Icoeff_new = λ1_new+Nf*(λ2_new-1)
                    Mat[Icoeff_new, Icoeff] = 1
                end
            end
        end
    end

    # Next J elements are λϕ, then J elements ϕλ
    for j = 1:Nj
        for l = 1:Nl
            λ      = ψ_ind[j,l]
            Icoeff = λ+Nf*(ϕ_ind-1)  # λϕ

            λ_new      = ψ_ind[j,mod1(l+l_shift,Nl)]
            Icoeff_new = λ_new+Nf*(ϕ_ind-1)  # λϕ

            Mat[Icoeff_new, Icoeff] = 1

            Icoeff = ϕ_ind+Nf*(λ-1)  # ϕλ
            Icoeff_new = ϕ_ind+Nf*(λ_new-1)  # ϕλ
            Mat[Icoeff_new, Icoeff] = 1
        end
    end

    # Next 1 element is ϕϕ
    Icoeff = ϕ_ind+Nf*(ϕ_ind-1)
    Mat[Icoeff, Icoeff] = 1

    # If the Omega filter exists, add more terms
    if Omega
        # Next J elements are λΩ, then J elements Ωλ
        for j = 1:Nj
            for l = 1:Nl
                λ      = ψ_ind[j,l]
                λ_new      = ψ_ind[j,mod1(l+l_shift,Nl)]
                Icoeff = λ+Nf*(Ω_ind-1)  # λΩ
                Icoeff_new = λ_new+Nf*(Ω_ind-1)  # λΩ
                Mat[Icoeff_new, Icoeff] = 1

                Iiso   = I0+Nj+j
                Icoeff = Ω_ind+Nf*(λ-1)  # Ωλ
                Icoeff_new = Ω_ind+Nf*(λ_new-1)  # Ωλ
                Mat[Icoeff_new, Icoeff] = 1
            end
        end
        # Next 3 elements are ϕΩ, Ωϕ, ΩΩ
        Mat[ϕ_ind+Nf*(Ω_ind-1), ϕ_ind+Nf*(Ω_ind-1)] = 1
        Mat[Ω_ind+Nf*(ϕ_ind-1), Ω_ind+Nf*(ϕ_ind-1)] = 1
        Mat[Ω_ind+Nf*(Ω_ind-1), Ω_ind+Nf*(Ω_ind-1)] = 1
    end

    return sparse(Mat)
end

function transformMaker(coeff, S1Mat, S2Mat)
    NS1 = size(S1Mat)[2]
    NS2 = size(S2Mat)[2]
    S1iso = transpose(S1Mat*transpose(coeff[:,2+1:2+NS1]))
    S2iso = transpose(S2Mat*transpose(coeff[:,2+NS1+1:2+NS1+NS2]))
    return hcat(coeff[:,1:2],S1iso,S2iso)
end

<PyCall.jlwrap transformMaker>

Do 3

In [18]:
L=3
filename = 'equiv_workup_L3.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L3_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L3_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L3_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L3_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [29]:
NTrainSubset_list = [2,4,8,12,16,20,24]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [01:41<00:00, 10.17s/it]


In [30]:
NTrainSubset_list = [2,4,8,12,16]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 5/5 [05:55<00:00, 71.19s/it]


In [31]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [00:47<00:00,  1.99s/it]


In [32]:
%%julia
L=3
mnist_DHC_out_test = h5read("mnist_DHC_test_eq_L3_s.h5","main/data")
filter_hash = DHC_2DUtils.fink_filter_hash(1, L, nx=128, pc=1, wd=2)
sizeTarget = size(transpose(mnist_DHC_out_test))
shifted_coeff = zeros(L,sizeTarget[1],sizeTarget[2])
for i=1:L
    S1mat = S1_equiv_matrix(filter_hash,i)
    S2mat = S2_equiv_matrix(filter_hash,i)
    shifted_coeff[i,:,:] = transformMaker(transpose(mnist_DHC_out_test),S1mat,S2mat)
end

In [33]:
shifted_coeff = np.array(Main.eval("shifted_coeff"))

In [34]:
equiv_std = np.zeros((len(angle_test),2))

x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val)))
    equiv_std[j,1] = np.mean(np.abs(diff))
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [06:33<00:00, 1218.30it/s]


In [19]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L3_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [20]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=3
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [07:28<00:00, 1069.19it/s]


Do 4

In [21]:
L=4
filename = 'equiv_workup_L4.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L4_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L4_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L4_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L4_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [40]:
NTrainSubset_list = [2,4,8,12,16,20,24]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [03:03<00:00, 18.37s/it]


In [41]:
NTrainSubset_list = [2,4,8,12,16]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 5/5 [10:25<00:00, 125.14s/it]


In [42]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [01:25<00:00,  3.57s/it]


In [43]:
%%julia
L=4
mnist_DHC_out_test = h5read("mnist_DHC_test_eq_L4_s.h5","main/data")
filter_hash = DHC_2DUtils.fink_filter_hash(1, L, nx=128, pc=1, wd=2)
sizeTarget = size(transpose(mnist_DHC_out_test))
shifted_coeff = zeros(L,sizeTarget[1],sizeTarget[2])
for i=1:L
    S1mat = S1_equiv_matrix(filter_hash,i)
    S2mat = S2_equiv_matrix(filter_hash,i)
    shifted_coeff[i,:,:] = transformMaker(transpose(mnist_DHC_out_test),S1mat,S2mat)
end

In [44]:
shifted_coeff = np.array(Main.eval("shifted_coeff"))

In [45]:
equiv_std = np.zeros((len(angle_test),2))

x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val)))
    equiv_std[j,1] = np.mean(np.abs(diff))
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [08:23<00:00, 952.50it/s]


In [22]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L4_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [23]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=4
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [09:48<00:00, 815.79it/s]


Do 5

In [39]:
L=5
filename = 'equiv_workup_L5.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L5_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L5_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L5_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L5_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [47]:
NTrainSubset_list = [2,4,8,12,16,20,24]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [05:16<00:00, 31.63s/it]


In [48]:
NTrainSubset_list = [2,4,8,12,16]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 5/5 [23:11<00:00, 278.25s/it]


In [49]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [02:25<00:00,  6.07s/it]


In [50]:
%%julia
L=5
mnist_DHC_out_test = h5read("mnist_DHC_test_eq_L5_s.h5","main/data")
filter_hash = DHC_2DUtils.fink_filter_hash(1, L, nx=128, pc=1, wd=2)
sizeTarget = size(transpose(mnist_DHC_out_test))
shifted_coeff = zeros(L,sizeTarget[1],sizeTarget[2])
for i=1:L
    S1mat = S1_equiv_matrix(filter_hash,i)
    S2mat = S2_equiv_matrix(filter_hash,i)
    shifted_coeff[i,:,:] = transformMaker(transpose(mnist_DHC_out_test),S1mat,S2mat)
end

In [51]:
shifted_coeff = np.array(Main.eval("shifted_coeff"))

In [52]:
equiv_std = np.zeros((len(angle_test),2))

x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val)))
    equiv_std[j,1] = np.mean(np.abs(diff))
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [10:57<00:00, 729.85it/s]


In [40]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L5_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [41]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=5
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [12:52<00:00, 621.67it/s]


OSError: Unable to create link (name already exists)

Do 6

In [27]:
L=6
filename = 'equiv_workup_L6.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L6_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L6_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L6_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L6_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [8]:
NTrainSubset_list = [2,4,8,12,16,20,24]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [07:42<00:00, 46.30s/it]


In [9]:
NTrainSubset_list = [2,4,8,12,16]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 5/5 [27:03<00:00, 324.67s/it]


In [10]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [03:42<00:00,  9.27s/it]


In [11]:
%%julia
L=6
mnist_DHC_out_test = h5read("mnist_DHC_test_eq_L6_s.h5","main/data")
filter_hash = DHC_2DUtils.fink_filter_hash(1, L, nx=128, pc=1, wd=2)
sizeTarget = size(transpose(mnist_DHC_out_test))
shifted_coeff = zeros(L,sizeTarget[1],sizeTarget[2])
for i=1:L
    S1mat = S1_equiv_matrix(filter_hash,i)
    S2mat = S2_equiv_matrix(filter_hash,i)
    shifted_coeff[i,:,:] = transformMaker(transpose(mnist_DHC_out_test),S1mat,S2mat)
end

In [12]:
shifted_coeff = np.array(Main.eval("shifted_coeff"))

In [13]:
equiv_std = np.zeros((len(angle_test),2))

x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val)))
    equiv_std[j,1] = np.mean(np.abs(diff))
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [14:30<00:00, 551.50it/s]


In [28]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L6_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [29]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=6
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [16:50<00:00, 474.86it/s]


Do 7

In [30]:
L=7
filename = 'equiv_workup_L7.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L7_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L7_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L7_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L7_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [15]:
NTrainSubset_list = [2,4,8,12,16,20,24]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [11:24<00:00, 68.43s/it]


In [16]:
NTrainSubset_list = [2,4,8,12]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 4/4 [34:43<00:00, 520.93s/it]


In [17]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [07:26<00:00, 18.59s/it]


In [31]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L7_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [11]:
equiv_std = np.zeros((len(angle_test),2))

x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val)))
    equiv_std[j,1] = np.mean(np.abs(diff))
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [17:52<00:00, 447.65it/s]


In [32]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=7
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [21:58<00:00, 364.13it/s]


Now 8

In [ ]:
L=8
filename = 'equiv_workup_L8.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L8_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L8_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L8_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L8_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [8]:
NTrainSubset_list = [2,4,8,12,16,20,24]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [15:48<00:00, 94.89s/it]


In [13]:
NTrainSubset_list = [2,4,8,12]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 4/4 [44:18<00:00, 664.54s/it]


In [14]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [08:07<00:00, 20.30s/it]


In [5]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L8_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [16]:
equiv_std = np.zeros((len(angle_test),2))

x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val)))
    equiv_std[j,1] = np.mean(np.abs(diff))
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [22:48<00:00, 350.62it/s]


In [35]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=8
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [28:38<00:00, 279.37it/s]


In [6]:
J=5
L=8
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
j=64
k = np.floor_divide(j,48)
x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
x_interp = np.append(x_interp,180+x_interp[0])
y_interp = shifted_coeff[:,j,:]
y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
ext_y = f(x_for_compare)
real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
diff = ext_y-real_y

In [8]:
hf = h5py.File('equivFit.h5', 'w')
hf.create_dataset('ext_y', data=ext_y)
hf.create_dataset('real_y', data=real_y)
hf.create_dataset('y_interp', data=y_interp)
hf.create_dataset('x_interp', data=x_interp)
hf.create_dataset('x_for_compare', data=x_for_compare)
hf.create_dataset('shifted_coeff', data=shifted_coeff[:,48*k:48*(k+1),:])
hf.close()

Do 9

In [5]:
L=9
filename = 'equiv_workup_L9.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L9_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L9_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L9_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L9_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [37]:
NTrainSubset_list = [2,4,8,12]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [15:41<00:00, 94.15s/it]


In [6]:
NTrainSubset_list = [2,4,8]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 3/3 [34:54<00:00, 698.21s/it]


In [7]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [15:00<00:00, 37.53s/it]


In [8]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L9_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [9]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=9
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [35:31<00:00, 225.14it/s]


Do 10

In [4]:
L=10
filename = 'equiv_workup_L10.h5'
mnist_DHC_out_train = hd5_open('mnist_DHC_train_eq_L10_s.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_test_eq_L10_s.h5','main/data')
mnist_DHC_out_train_angles = hd5_open('mnist_DHC_train_eq_L10_s.h5','main/angles')
mnist_DHC_out_test_angles = hd5_open('mnist_DHC_test_eq_L10_s.h5','main/angles')

NtestAngle = mnist_DHC_out_test_angles.shape[0]
NtrainAngle = mnist_DHC_out_train_angles.shape[0]

angle_test = np.tile(mnist_DHC_out_test_angles,10000)
angle_test_class = np.repeat(mnist_test_y,NtestAngle)

angle_train = np.tile(mnist_DHC_out_train_angles,60000)
angle_train_class = np.repeat(mnist_train_y,NtrainAngle)

lb = LabelBinarizer()
train_y_onehot = lb.fit_transform(angle_train_class)

lb = LabelBinarizer()
test_y_onehot = lb.fit_transform(angle_test_class)

In [5]:
NTrainSubset_list = [2,4,8]
std_class = np.zeros((len(NTrainSubset_list),10,5))
for i in range(len(NTrainSubset_list)):
    for j in tqdm(range(10)):
        NTrainSubset = NTrainSubset_list[i]
        idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)
        
        sc = StandardScaler()
        tr_slice = np.logical_and(np.isin(angle_train,mnist_DHC_out_train_angles[idx]),np.isin(angle_train_class,j))
        tst_slice = np.isin(angle_test_class,j)
        
        X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_trx = np.array([
            np.cos(2*np.mod(angle_train[tr_slice],np.pi))
        ]).T

        X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testx = np.array([
            np.cos(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_trx, Y_trx)
        y_pred_testx = reg.predict(X_testx)
        y_pred_trainx = reg.predict(X_trx)

        train_lossl2x = np.mean(np.square(y_pred_trainx-Y_trx))
        test_lossl2x = np.mean(np.square(y_pred_testx-Y_testx))

        X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
        Y_try = np.array([
            np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
        ]).T

        X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
        Y_testy = np.array([
            np.sin(2*np.mod(angle_test[tst_slice],np.pi))
        ]).T

        reg = LinearRegression().fit(X_try, Y_try)
        y_pred_testy = reg.predict(X_testy)
        y_pred_trainy = reg.predict(X_try)

        train_lossl2y = np.mean(np.square(y_pred_trainy-Y_try))
        test_lossl2y = np.mean(np.square(y_pred_testy-Y_testy))
        
        std_class[i,j,0] = train_lossl2x
        std_class[i,j,1] = test_lossl2x
        
        std_class[i,j,2] = train_lossl2y
        std_class[i,j,3] = test_lossl2y
        
        diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
        diff[diff<-180]+=360
        diff[diff>180]-=360

        std_class[i,j,4] = np.std(diff)
        
hf = h5py.File(filename, 'w')
hf.create_dataset('std_class', data=std_class)
hf.close()

100%|██████████| 10/10 [12:16<00:00, 73.65s/it]


In [6]:
NTrainSubset_list = [2,4,8]
std_class_both = np.zeros((len(NTrainSubset_list),10,9))
std_class_both_angle = np.zeros((len(NTrainSubset_list),10,NtestAngle))
for i in tqdm(range(len(NTrainSubset_list))):
    NTrainSubset = NTrainSubset_list[i]
    idx = np.round(np.linspace(0, NtrainAngle - 1, NTrainSubset)).astype(int)

    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[idx])
    tst_slice = np.isin(angle_test_class,[0,1,2,3,4,5,6,7,8,9])

    X_trx = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trx = np.array([
        np.cos(2*np.mod(angle_train[tr_slice],np.pi))
    ]).T

    X_testx = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testx = np.array([
        np.cos(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_trx, Y_trx)
    y_pred_testx = reg.predict(X_testx)
    y_pred_trainx = reg.predict(X_trx)

    train_lossl2x = np.square(y_pred_trainx-Y_trx)
    test_lossl2x = np.square(y_pred_testx-Y_testx)

    X_try = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_try = np.array([
        np.sin(2*np.mod(angle_train[tr_slice],np.pi)),
    ]).T

    X_testy = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testy = np.array([
        np.sin(2*np.mod(angle_test[tst_slice],np.pi))
    ]).T

    reg = LinearRegression().fit(X_try, Y_try)
    y_pred_testy = reg.predict(X_testy)
    y_pred_trainy = reg.predict(X_try)

    train_lossl2y = np.square(y_pred_trainy-Y_try)
    test_lossl2y = np.square(y_pred_testy-Y_testy)
    
    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)

    diff = 180*(np.arctan2(Y_testy[:,0],Y_testx[:,0])/np.pi-np.arctan2(y_pred_testy[:,0],y_pred_testx[:,0])/np.pi)
    diff[diff<-180]+=360
    diff[diff>180]-=360

    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)
        
        for indxl, l in enumerate(mnist_DHC_out_test_angles):
            masktst_l = np.logical_and(np.isin(angle_test_class[tst_slice],j),np.isin(angle_test[tst_slice],l))
            std_class_both_angle[i,j,indxl] = np.count_nonzero(class_pred[masktst_l]==j)/np.count_nonzero(y_class[masktst_l]==j)
        
        std_class_both[i,j,0] = np.mean(train_lossl2x[masktr])
        std_class_both[i,j,1] = np.mean(test_lossl2x[masktst])
        std_class_both[i,j,2] = np.mean(train_lossl2y[masktr])
        std_class_both[i,j,3] = np.mean(test_lossl2y[masktst])  
        std_class_both[i,j,4] = np.mean(train_lossl2c[masktr,j])
        std_class_both[i,j,5] = np.mean(test_lossl2c[masktst,j])  
        std_class_both[i,j,6] = np.std(diff[masktst])
        std_class_both[i,j,7] = cmat[j,j]/np.count_nonzero(masktst)
        std_class_both[i,j,8] = np.std(std_class_both_angle[i,j,:])
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_both', data=std_class_both)
hf.close()

100%|██████████| 3/3 [49:35<00:00, 991.90s/it]


In [5]:
std_class_ref = np.zeros((NtrainAngle,10,3))
for i in tqdm(range(NtrainAngle)):
    sc = StandardScaler()
    tr_slice = np.isin(angle_train,mnist_DHC_out_train_angles[i])
    tst_slice = np.isin(angle_test,mnist_DHC_out_train_angles[i])

    X_trc = sc.fit_transform(np.log10(mnist_DHC_out_train[tr_slice]))
    Y_trc = train_y_onehot[tr_slice]

    X_testc = sc.transform(np.log10(mnist_DHC_out_test[tst_slice]))
    Y_testc = test_y_onehot[tst_slice]

    reg = LinearRegression().fit(X_trc, Y_trc)
    y_pred_testc = reg.predict(X_testc)
    y_pred_trainc = reg.predict(X_trc)

    train_lossl2c = np.square(y_pred_trainc-Y_trc)
    test_lossl2c = np.square(y_pred_testc-Y_testc)
    
    class_pred = np.argmax(y_pred_testc,axis=1)
    y_class = angle_test_class[tst_slice]
    cmat = cm_eval(class_pred,y_class,10)
    
    for j in range(10):
        masktst = np.isin(angle_test_class[tst_slice],j)
        masktr = np.isin(angle_train_class[tr_slice],j)

        std_class_ref[i,j,0] = np.mean(train_lossl2c[masktr,j])
        std_class_ref[i,j,1] = np.mean(test_lossl2c[masktst,j])  
        std_class_ref[i,j,2] = cmat[j,j]/np.count_nonzero(masktst)
        
hf = h5py.File(filename, 'a')
hf.create_dataset('std_class_ref', data=std_class_ref)
hf.close()

100%|██████████| 24/24 [15:30<00:00, 38.76s/it]


In [6]:
shifted_coeff = hd5_open("mnist_DHC_test_eq_L10_s1.h5","main/data")
shifted_coeff = np.transpose(shifted_coeff)

In [7]:
equiv_std = np.zeros((len(angle_test),8))
J=5
L=10
x_for_compare = mnist_DHC_out_test_angles*180/np.pi
for j in tqdm(range(len(angle_test))):
    k = np.floor_divide(j,48)
    x_interp = np.array([angle_test[j]+(i+1)*np.pi/L for i in range(L)])*180/np.pi
    x_interp = np.append(x_interp,180+x_interp[0])
    y_interp = shifted_coeff[:,j,:]
    y_interp = np.vstack((y_interp,y_interp[0,:].reshape(1,y_interp.shape[1])))
    f = interpolate.CubicSpline(x_interp,y_interp,axis=0,bc_type='periodic')
    ext_y = f(x_for_compare)
    real_y = mnist_DHC_out_test[48*k:48*(k+1),:]
    diff = ext_y-real_y
    max_val = np.amax([ext_y,real_y],axis=0)
    equiv_std[j,0] = np.mean(np.abs(np.divide(diff,max_val))[:,0])
    equiv_std[j,1] = np.mean(np.abs(diff)[:,0])
    equiv_std[j,2] = np.mean(np.abs(np.divide(diff,max_val))[:,1])
    equiv_std[j,3] = np.mean(np.abs(diff)[:,1])
    equiv_std[j,4] = np.mean(np.abs(np.divide(diff,max_val))[:,2:J*L+2])
    equiv_std[j,5] = np.mean(np.abs(diff)[:,2:J*L+2])
    equiv_std[j,6] = np.mean(np.abs(np.divide(diff,max_val))[:,J*L+3:])
    equiv_std[j,7] = np.mean(np.abs(diff)[:,J*L+3:])
    
hf = h5py.File(filename, 'a')
hf.create_dataset('equiv_std_full', data=equiv_std)
hf.close()

100%|██████████| 480000/480000 [43:00<00:00, 186.03it/s]


Can we do 12, 16, 32? Let's not. I think we have a clear enough trend. 